In [ ]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim  # optimizers e.g. gradient descent, ADAM, etc.
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
from sklearn.model_selection import train_test_split

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat
import glob
import os

import subprocess
import os
import cv2

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

In [ ]:
folder = './Training and Testing Data/'
list_of_files = ['3p0v.tdms','3p5v.tdms','4p0v.tdms','4p5v.tdms','5p0v.tdms','5p5v.tdms','6p0v.tdms','Thermal Cyc.tdms','Distance Cyc 4rotation.tdms','reference for all.tdms','1p5v.tdms','2p0v.tdms','2p5v.tdms']
shift_val = [80,90,80,85,86,84,88,80,80,0,81,86,81]

X_end_ref = np.array(pd.read_csv('END-REFLECTION.tdms.csv',delimiter=",",header = None))
X_end_ref = np.mean(X_end_ref,axis = 0)

X = []

i =0 
for label in list_of_files: 
    X_temp = np.array(pd.read_csv(label +'.csv',header = None))
    X_temp = X_temp#/X_end_ref
    X.append(X_temp[15+shift_val[i]:2015+shift_val[i]])
    i = i+1


In [ ]:
dir_name_list = ['./3p0v_r/','./3p5v_r/','./4p0v_r/','./4p5v_r/','./5p0v_r/','./5p5v_r/','./6p0v_r/','./Thermal Cyc 5.5 to 0 to 5.5_r/','./Distance Cyc 4 rotation_r/','./reference_r/','./1p5v_r/','./2p0v_r/','./2p5v_r/']
Y = []
for dir_name in dir_name_list:
    list_of_files = sorted(filter(os.path.isfile, glob.glob(dir_name + '*')))

    Y_temp = np.zeros((len(list_of_files),480))
    i = 0
    for file in list_of_files:
        Y_temp[i,:] = np.array(pd.read_csv(dir_name+'file_'+'%04d'%i+'.csv',sep=',',header = 0))[:,2]
        i = i+1
    print(dir_name)
    
    Y.append(Y_temp[:2000,:])
    # Y.append(Y_temp[:2000,188:292])

In [ ]:

X[0] = np.vstack((X[0][:115],X[0][171:1216],X[0][1275:]))#3.0
Y[0] = np.vstack((Y[0][:115],Y[0][171:1216],Y[0][1275:]))

X[1] = np.vstack((X[1][:168],X[1][225:1270],X[1][1327:]))#3.5
Y[1] = np.vstack((Y[1][:168],Y[1][225:1270],Y[1][1327:]))


X[2] = np.vstack((X[2][:171],X[2][228:1270],X[2][1327:]))#4.0
Y[2] = np.vstack((Y[2][:171],Y[2][228:1270],Y[2][1327:]))


X[3] = np.vstack((X[3][:117],X[3][174:1217],X[3][1277:]))# 4.5
Y[3] = np.vstack((Y[3][:117],Y[3][174:1217],Y[3][1277:]))


X[4] = np.vstack((X[4][:169],X[4][225:1464],X[4][1526:]))#5.0
Y[4] = np.vstack((Y[4][:169],Y[4][225:1464],Y[4][1526:]))


X[5] = np.vstack((X[5][:120],X[5][175:1220],X[5][1276:]))#5.5
Y[5] = np.vstack((Y[5][:120],Y[5][175:1220],Y[5][1276:]))


X[6] = np.vstack((X[6][:124],X[6][179:1225],X[6][1280:]))#6.0
Y[6] = np.vstack((Y[6][:124],Y[6][179:1225],Y[6][1280:]))

X[7] = np.vstack((X[7][:167],X[7][225:1369],X[7][1427:])) #Thermal Cycle
Y[7] = np.vstack((Y[7][:167],Y[7][225:1369],Y[7][1427:]))

X[8] = np.vstack((X[8][:166],X[8][224:1767],X[8][1823:]))#Distance Cycle
Y[8] = np.vstack((Y[8][:166],Y[8][224:1767],Y[8][1823:]))

X[10] = np.vstack((X[10][:169],X[10][225:1369],X[10][1428:]))#1.5
Y[10] = np.vstack((Y[10][:169],Y[10][225:1369],Y[10][1428:]))

X[11] = np.vstack((X[11][:120],X[11][176:1220],X[11][1278:]))#2.0
Y[11] = np.vstack((Y[11][:120],Y[11][176:1220],Y[11][1278:]))

X[12] = np.vstack((X[12][:120],X[12][175:1220],X[12][1276:]))#2.5
Y[12] = np.vstack((Y[12][:120],Y[12][175:1220],Y[12][1276:]))


In [ ]:
X_full = X[0]
Y_full = Y[0]


for i in range(1,len(X)):
    X_full = np.vstack((X_full,X[i]))
    Y_full = np.vstack((Y_full,Y[i]))

In [ ]:
X_full = torch.tensor(X_full,device = device,dtype = torch.float)
Y_full = torch.tensor(Y_full,device = device,dtype = torch.float)

In [ ]:
X_mean = torch.mean(X_full, axis = 0)
X_std = torch.std(X_full, axis = 0)
X_full = (X_full - X_mean)/X_std

Y_mean = torch.mean(Y_full, axis = 0)
Y_std = torch.std(Y_full, axis = 0)
Y_full = (Y_full - Y_mean)/Y_std

In [ ]:
#X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.0001, random_state=1)
train_size = X_full.shape[0]
batches = 50

batch_size = int(train_size/batches)

x_train_full = []
y_train_full = []

for b in range(batches):
    start = b*batch_size
    end = b*batch_size + batch_size
    x_train_full.append(X_full[start:end])
    y_train_full.append(Y_full[start:end])
    
del X,Y

In [ ]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        #self.activation = nn.Tanh()
        self.activation = nn.ReLU()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 

    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                    
    def loss(self,x_train,y_train):

        loss1 = self.loss_function(self.forward(x_train), y_train)
        
        return loss1

In [ ]:
def train_step(seed,x_train,y_train):
    
    optimizer.zero_grad()     # zeroes the gradient buffers of all parameters
    loss = NN.loss(x_train,y_train)
    loss.backward() #backprop
    optimizer.step()
    
    return loss

In [ ]:
def train_model(max_iter,rep, batches): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0

    for i in range(max_iter):
        for b in range(batches):
            x_train = x_train_full[b]
            y_train = y_train_full[b]
            loss_np = train_step(9*i+6*b,x_train,y_train).cpu().detach().numpy()
            #loss_val = NN.loss(X_val,Y_val).cpu().detach().numpy()
        if(i%100==0):
            print(i,"Train Loss",loss_np)#" Validation Loss:",loss_val)
        train_loss_history.append(loss_np)
    
    print(i,"Train Loss",loss_np)#" Validation Loss:",loss_val)
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [ ]:
max_reps = 1
max_iter = 5000

train_loss_full = []
test_mse_full = []
test_re_full = []

elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

train_loss_history = []
for reps in range(max_reps):  
   
    train_loss = []
    test_mse_loss = []

    torch.manual_seed(reps*36)

    layers = np.array([800,700,600,500,480])

    NN = Sequentialmodel(layers)
    
    NN.to(device)

    
    print(NN)

    params = list(NN.parameters())

    optimizer = torch.optim.SGD(NN.parameters(), lr=8e-2)


    train_model(max_iter,reps,batches)
    torch.save(NN.state_dict(),'Full_data_final_Jan9_'+str(int(time.time()))+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    
    print('Training time: %.2f' % (elapsed_time[reps]))

In [ ]:
folder = './Training and Testing Data/'

list_of_files = ['5p3v.tdms']

shift_val = [84]

X_test = []

i =0

for label in list_of_files: 
    X_temp = np.array(pd.read_csv(label +'.csv',header = None))
    X_temp = (X_temp)#/(X_end_ref)
    X_test.append(X_temp[15+shift_val[i]:2015+shift_val[i]])
    i = i+1


In [ ]:
dir_name_list = ['./5p3v_r/']
Y_test = []
for dir_name in dir_name_list:
    list_of_files = sorted(filter(os.path.isfile, glob.glob(dir_name + '*')))

    Y_temp = np.zeros((len(list_of_files),480))
    i = 0
    for file in list_of_files:
        Y_temp[i,:] = np.array(pd.read_csv(dir_name+'file_'+'%04d'%i+'.csv',sep=',',header = 0))[:,2]
        i = i+1
    print(dir_name)
    
    Y_test.append(Y_temp[:2000,:])

In [ ]:
layers = np.array([800,700,600,500,480])
NN = Sequentialmodel(layers)
    
NN.to(device)
NN.load_state_dict(torch.load('Full_data_final_Jan9_1673315453.pt', map_location=device))

In [ ]:
X_full_test = X_test[0]
Y_full_test = Y_test[0]


for i in range(1,len(X_test)):
    X_full_test = np.vstack((X_full_test,X_test[i]))
    Y_full_test = np.vstack((Y_full_test,Y_test[i]))
    
X_full_test = torch.tensor(X_full_test,device = device,dtype = torch.float)
Y_full_test = torch.tensor(Y_full_test,device = device,dtype = torch.float)

In [ ]:
X_full_test = (X_full_test - X_mean)/X_std

In [ ]:
Y_pred_full = np.zeros((2000,480))
for i in range(2000):
    Y_pred_full[i,:]= (NN.forward(X_full_test[i,:])*Y_std + Y_mean).cpu().detach().numpy()

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(Y_pred_full,cmap = 'jet',interpolation='none',extent=[0,13.824,2000,0],aspect = 0.0144,vmin = 0,vmax = 800)

ax.set_xticks(np.linspace(0,480*28.8/1000,4), minor=False)
ax.tick_params(axis = 'both',bottom = True, top = False, labelbottom = True,labeltop = False,direction = 'out',labelsize = 8)
ax.set_xlabel('Location ($mm$)',fontsize = 10)
ax.set_ylabel('Time ($ms$)',fontsize = 10)
ax.xaxis.set_label_position('bottom') 
ax.set_title('5.3V Case Predicted',fontweight = 600)

In [ ]:
for i in range(10):
    plt.subplots(1,1)
    Y_pred_test= (NN.forward(X_full_test[200*i,:])*Y_std + Y_mean).cpu().detach().numpy()
    Y_true_test = Y_full_test[200*i,:].cpu().detach().numpy()
    plt.plot(188*np.ones(2,),[0,800],'m--')
    plt.plot(292*np.ones(2,),[0,800],'m--')
    plt.plot(Y_pred_test,'b',label = 'Predicted')
    plt.plot(Y_true_test,'r',label = 'True')
    plt.xlabel('x',fontsize =18)
    plt.ylabel('T',fontsize =18)
    plt.title('Frame'+str(200*i))
    plt.xlim([0,480])
    plt.ylim([0,800])

In [ ]:
min_test = 400
max_test = 1600
iou = np.zeros((max_test - min_test,1))

for k in range(min_test,max_test):
    i = k-min_test
    Y_pred_val= (NN.forward(X_full_test[k,:])*Y_std + Y_mean).cpu().detach().numpy()
    Y_true_val = Y_full_test[k,:].cpu().detach().numpy()
    
    iou_1 = np.sum(np.maximum(Y_pred_val,Y_true_val))
    iou_2 = np.sum(np.minimum(Y_pred_val,Y_true_val))
    
    iou[i] = iou_2/iou_1
    
print('IOU : ',np.mean(iou))

In [ ]:
#Revision Modification Jan 2024
mean_ae= np.zeros((max_test - min_test,1))
for k in range(min_test,max_test):
    i = k-min_test
    Y_pred_val= (NN.forward(X_full_test[k,:])*Y_std + Y_mean).cpu().detach().numpy()
    Y_true_val = Y_full_test[k,:].cpu().detach().numpy()
    
    # iou_1 = np.sum(np.maximum(Y_pred_val,Y_true_val))
    # iou_2 = np.sum(np.minimum(Y_pred_val,Y_true_val))
    mean_ae[i] = np.mean(np.abs(Y_pred_val-Y_true_val)) 
    
    # iou[i] = iou_2/iou_1
    
print('Max AE : ',np.max(mean_ae))

In [ ]:
min_test = 400
max_test = 1600
iou = np.zeros((max_test - min_test,1))

for k in range(min_test,max_test):
    i = k-min_test
    Y_pred_val= (NN.forward(X_full_test[k,:])*Y_std + Y_mean).cpu().detach().numpy()
    Y_true_val = Y_full_test[k,:].cpu().detach().numpy()
    
    iou_1 = np.sum(np.maximum(Y_pred_val[188:292],Y_true_val[188:292]))
    iou_2 = np.sum(np.minimum(Y_pred_val[188:292],Y_true_val[188:292]))
    
    iou[i] = iou_2/iou_1
    
print('IOU : ',np.mean(iou))

In [ ]:
min_test = 400
max_test = 1600
mean_ae= np.zeros((max_test - min_test,1))

for k in range(min_test,max_test):
    i = k-min_test
    Y_pred_val= (NN.forward(X_full_test[k,:])*Y_std + Y_mean).cpu().detach().numpy()
    Y_true_val = Y_full_test[k,:].cpu().detach().numpy()
    
    # iou_1 = np.sum(np.maximum(Y_pred_val[188:292],Y_true_val[188:292]))
    # iou_2 = np.sum(np.minimum(Y_pred_val[188:292],Y_true_val[188:292]))
    mean_ae[i] = np.mean(np.abs(Y_pred_val[188:292]-Y_true_val[188:292])) 
    
    # iou[i] = iou_2/iou_1
    
print('Max AE : ',np.max(mean_ae))

In [ ]:
c = 0
for k in range(min_test,max_test):
    c = c + np.corrcoef(NN.forward(X_full_test[k,:]).cpu().detach().numpy(),Y_full_test[k,:].cpu().detach().numpy())

In [ ]:
Y_pred_corr = []
Y_true_corr  = []
for k in range(min_test,max_test):
    Y_pred_val= Y_pred_corr.append((NN.forward(X_full_test[k,:])*Y_std + Y_mean).cpu().detach().numpy())
    Y_true_val = Y_true_corr.append(Y_full_test[k,:].cpu().detach().numpy())

In [ ]:
Y_pred_corr = np.array(Y_pred_corr).reshape(-1,1)
Y_true_corr = np.array(Y_true_corr).reshape(-1,1)

In [ ]:
idx = np.random.randint(low=0, high=57600, size=(5760,))

In [ ]:
 np.corrcoef(Y_pred_corr[idx].reshape(-1,),Y_true_corr[idx].reshape(-1,))

In [ ]:
Y_true_corr[idx].shape

In [ ]:
folder = './XXXX/'
fig,ax = plt.subplots()
ax.plot(Y_pred_corr[idx],Y_true_corr[idx],'r*',markersize = 1,label = 'Data Points')
ax.plot([0,800],[0,800],'k',label = 'Ideal Reference')
ax.set_xlim([0,800])
ax.set_ylim([0,800])

ax.set_xlabel('Predicted Temperature ($^\circ$$C$)',fontsize = 10)
ax.set_ylabel('Actual Temperature ($^\circ$$C$)',fontsize = 10)
ax.set_aspect(aspect = 0.9)
ax.text(400,300,'Correlation = 0.994')

ax.set_title('Correlation Plot')
ax.legend()
plt.savefig(folder+'5p3v_correlation.png',format = 'png',pad_inches=0, bbox_inches='tight')

In [ ]:
list_of_files = ['170-800-200.tdms']

X_test = []

i =0

for label in list_of_files: 
    X_temp = np.array(pd.read_csv(label +'_smooth_lr21.csv',header = None))
    X_ref = np.mean(X_temp[:400],axis =0)
    X_temp = 1.5*X_temp#/X_end_ref
    i = i+1
    
X_temp = torch.tensor(X_temp,device = device,dtype = torch.float)
X_temp = (X_temp - X_mean)/X_std

In [ ]:
Y_pred_test = np.zeros((9000,480))
for i in range(9000):
    Y_pred_test[i,:]= (NN.forward(X_temp[i,:])*Y_std + Y_mean).cpu().detach().numpy()
Y_pred_mean = np.mean(Y_pred_test[:200,:],axis = 0)

In [ ]:
# # Y_250 = NN.forward(X_temp[250,:])*Y_std + Y_mean
# for i in range(20):
#     plt.subplots(1,1)
#     idx = 1500+100*i
#     Y_pred_test= (NN.forward(X_temp[idx,:])*Y_std + Y_mean ).cpu().detach().numpy() - Y_pred_mean + 27
#     plt.plot(188*np.ones(2,),[0,800],'m--')
#     plt.plot(292*np.ones(2,),[0,800],'m--')
#     plt.plot(Y_pred_test,'b',label = 'Predicted')
#     plt.xlabel('x',fontsize =18)
#     plt.ylabel('T',fontsize =18)
#     plt.title('Frame'+str(idx))
#     plt.xlim([0,480])
#     plt.ylim([0,800])